# BERT 2 TWEETS

# Train : Set-1 (50 % )
# Val   : 
> # Set-2 (25 %)
> # Set-3 (25 %)
> # Test


In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q ktrain 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import ktrain
from ktrain import text 

In [ ]:
import tensorflow as tf
import numpy as np 
import pandas as pd 

In [ ]:
set1 = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/set1.csv')
set2 = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/set2.csv')
set3 = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/set3.csv')

test = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/tweets_test_features.csv')

In [ ]:
train = set1.copy()
val = set2.copy()

x2D = set2.drop('target',axis=1)
x3D = set3.drop('target',axis=1)

# Perform One hot encoding of target column

In [ ]:
train = train.rename(columns = {'target' : 'Disaster'})
train['not_a_disaster'] = train['Disaster'].apply(lambda x : 0 if x==1 else 1)

In [ ]:
val = val.rename(columns = {'target' : 'Disaster'})
val['not_a_disaster'] = val['Disaster'].apply(lambda x : 0 if x==1 else 1)

# Preprocess

In [ ]:
(x_tr,y_tr),(x_te,y_te),preprocess = text.texts_from_df(train_df = train,
                                                        text_column = 'text',
                                                        label_columns = ['Disaster','not_a_disaster'],  
                                                        val_df = val,             
                                                        maxlen = 500,
                                                        preprocess_mode = 'bert',                                                       
                                                        ngram_range=3                                                                                                               
                                                        )

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
model = text.text_classifier(name = 'bert',
                             train_data = (x_tr, y_tr),
                             preproc = preprocess
                             )

Is Multi-Label? False
maxlen is 500
done.


In [ ]:
learner = ktrain.get_learner(model = model,
                             train_data = (x_tr,y_tr),
                             val_data = (x_te, y_te),
                             batch_size = 6
                             ) 
#val_data = (x_te, y_te),

In [ ]:
learner.fit_onecycle(lr = 2e-5, epochs = 2 )
#learner.autofit(lr = 2e-5)



begin training using onecycle policy with max lr of 2e-05...
Train on 3806 samples, validate on 1903 samples
Epoch 1/2
3806/3806 [==============================] - 493s 130ms/sample - loss: 0.4938 - accuracy: 0.7609 - val_loss: 0.4170 - val_accuracy: 0.8213
Epoch 2/2
3806/3806 [==============================] - 477s 125ms/sample - loss: 0.3290 - accuracy: 0.8699 - val_loss: 0.4154 - val_accuracy: 0.8234


In [ ]:
learner.validate(val_data = (x_te,y_te), class_names = ['Disaster','not_a_disaster'])

                precision    recall  f1-score   support

      Disaster       0.79      0.80      0.79       817
not_a_disaster       0.85      0.84      0.85      1086

      accuracy                           0.82      1903
     macro avg       0.82      0.82      0.82      1903
  weighted avg       0.82      0.82      0.82      1903



array([[651, 166],
       [170, 916]])

In [ ]:
abc

# Predict_Proba

In [ ]:
predictor = ktrain.get_predictor(learner.model, preprocess)

In [ ]:
#test_data = x3D.text.values 
x2 = x2D.text.values
x3 = x3D.text.values
test_data = test.text.values

In [ ]:
x2_proba_arr = predictor.predict(x2, return_proba=True)

In [ ]:
x3_proba_arr = predictor.predict(x3, return_proba=True)

In [ ]:
test_proba_arr = predictor.predict(test_data, return_proba=True)

In [ ]:
x2_proba_list = x2_proba_arr.tolist()
x3_proba_list = x3_proba_arr.tolist()
test_proba_list = test_proba_arr.tolist()

# Probability Dataframes

In [ ]:
disaster_probability = []
not_a_disaster_probability = []
for i in x2_proba_list:
  disaster_probability.append(i[0])
  not_a_disaster_probability.append(i[1])

x2_proba_df = pd.DataFrame({'Bert_1': disaster_probability, 'Bert_0': not_a_disaster_probability})

In [ ]:
disaster_probability = []
not_a_disaster_probability = []
for i in x3_proba_list:
  disaster_probability.append(i[0])
  not_a_disaster_probability.append(i[1])

x3_proba_df = pd.DataFrame({'Bert_1': disaster_probability, 'Bert_0': not_a_disaster_probability})

In [ ]:
disaster_probability = []
not_a_disaster_probability = []
for i in test_proba_list:
  disaster_probability.append(i[0])
  not_a_disaster_probability.append(i[1])

test_proba_df = pd.DataFrame({'Bert_1': disaster_probability, 'Bert_0': not_a_disaster_probability})

# Save Probas to drive

In [ ]:
x2_proba_df.to_csv('set2_Bert_proba.csv', index=False) 
!cp set2_Bert_proba.csv "/content/drive/My Drive/Tweets/5. Predict Probas" 

x3_proba_df.to_csv('set3_Bert_proba.csv', index=False) 
!cp set3_Bert_proba.csv "/content/drive/My Drive/Tweets/5. Predict Probas" 

test_proba_df.to_csv('test_data_Bert_proba.csv', index=False) 
!cp test_data_Bert_proba.csv "/content/drive/My Drive/Tweets/5. Predict Probas" 

# Sample of the files saved

In [ ]:
x2_proba_df[:2]

,Bert_1,Bert_0
0,0.388212,0.611788
1,0.985258,0.014742


In [ ]:
x3_proba_df[:2]

,Bert_1,Bert_0
0,0.057976,0.942024
1,0.274469,0.725531


In [ ]:
test_proba_df[:2]

,Bert_1,Bert_0
0,0.954075,0.045925
1,0.965994,0.034006
